In [ ]:
import math
import sys
import os

import IPython
import matplotlib.pylab as plt
import numpy as np
from scipy import signal

%load_ext autoreload
%autoreload 2

In [ ]:
def print_result(key, bins, t_sec=1, end='STOP', start_longer=True):
    # 1Q per beat (quarter note = beat)
    # bps = r(bpm) / 60 (beat per second)
    # => seconds per Q: t = 60/r
    r = round(60 / t_sec)

    print(f"static Melody {key}" + r" = {.bpm = " + f"{int(r)}" + r", .delay = 1, .notes = {", end="")
    duration = 0
    for i, freq in enumerate(freqs[bins]):
        if (start_longer) and (i == 0):
            # add longer duration in beginning to allow for experiment
            # to start
            print(r"{" + f"{int(round(freq))}, W" + r"}, ", end="")
            duration += t_sec * 4
        else:
            print(r"{" + f"{int(round(freq))}, Q" + r"}, ", end="")
            duration += t_sec
    print(end + r"}};" +" \n")
    #print(f'duration: {duration:.0f} seconds, {duration/60:.0f} minutes \n')

In [ ]:
firmware_dir = "/home/duembgen/Documents/PhD/workspace/crazyflie-firmware/"
out_name = firmware_dir + "src/modules/src/sound_cf2.c"

In [ ]:
from bin_selection import generate_sweep
from crazyflie_description_py.parameters import SOUND_EFFECTS, N_BUFFER, FS

freqs = np.fft.rfftfreq(N_BUFFER, 1/FS)

print(f"add the following in {out_name} around line 200: \n")

for key in SOUND_EFFECTS.keys():
    if 'mono' in key:
        continue
        
    bins, t_sec = generate_sweep(key)
    
    end = 'STOP'
    start_longer = True
    if key == 'sweep_slow':
        start_longer = False
        end = 'REPEAT'
    elif key == 'sweep_fast':
        start_longer = False
        end = 'REPEAT'
        
    bins = np.array(bins)
    bins = bins[bins>0]
    print_result(key, bins, t_sec=t_sec, end=end, start_longer=start_longer)

In [ ]:
n_freq=N_BUFFER//2+1
fs1 = 32000
fs2 = 44100

delta = 1000
fmin = 1000
fmax = fmin + delta

f1 = np.fft.rfftfreq(N_BUFFER, 1/fs1)
f2 = np.fft.rfftfreq(round(N_BUFFER*fs2/fs1), 1/fs2)
f1_mask = f1[(f1>=fmin) & (f1<fmax)]
f2_mask = f2[(f2>=fmin) & (f2<fmax)]
print('errors:', f2_mask[:5]-f1_mask[:5])
fig = plt.figure()
fig.set_size_inches(15, 5)
[plt.plot([f, f], [0, 1], color='C0') for f in f1_mask]
[plt.plot([f, f], [1, 2], color='C1') for f in f2_mask]
pass                                                        